# Importing the necessary library

# the model predicts only the first innings data

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import numpy as np
import pandas as pd
from yaml import safe_load
import os
from tqdm import tqdm
import chardet
import pickle

In [3]:
# Create filenames

# filenames = []
# for file in os.listdir('/content/drive/MyDrive/Projects/data'):
#     filenames.append(os.path.join('/content/drive/MyDrive/Projects/data', file)) # Add the missing portion of the path

filenames = []
for file in os.listdir('/Users/kaushalsingh/Desktop/Projects/t20i-score-predictor/data'):
    filenames.append(os.path.join('/Users/kaushalsingh/Desktop/Projects/t20i-score-predictor/data', file)) # Add the missing portion of the path


In [4]:
def detect_encoding(file):
    with open(file, 'rb') as f:
        result = chardet.detect(f.read())
        return result['encoding']

final_df = pd.DataFrame()

counter = 1
dataframes = []
for file in tqdm(filenames):
    if file.endswith('.json') or file.endswith('.yaml'):  # Add condition to process only JSON or YAML files
        encoding = detect_encoding(file)
        try:
            with open(file, 'r', encoding=encoding) as f:
                df = pd.json_normalize(safe_load(f))
                df['match_id'] = counter
                counter += 1
                dataframes.append(df)
        except Exception as e:
            print(f"Error processing file {file}: {e}")

final_df = pd.concat(dataframes, ignore_index=True)

# final_df

100%|██████████| 1434/1434 [02:01<00:00, 11.78it/s]


In [5]:
backup = final_df

In [6]:
columns_to_drop = [
    'meta.data_version',
    'meta.created',
    'meta.revision',
    'info.outcome.bowl_out',
    'info.supersubs.South Africa',
    'info.supersubs.New Zealand',
    'info.outcome.eliminator',
    'info.outcome.result',
    'info.outcome.method',
    'info.neutral_venue',
    'info.match_type_number',
    'info.outcome.by.runs',
    'info.outcome.by.wickets',
    'info.dates',
    'info.player_of_match',
    'info.umpires',
    'info.bowl_out',
    'info.toss.decision',
    'info.toss.winner'
]

# Filter out columns that are not present in the DataFrame
existing_columns_to_drop = [col for col in columns_to_drop if col in final_df.columns]

final_df.drop(columns=existing_columns_to_drop, inplace=True)


In [7]:
final_df

,innings,info.city,info.gender,info.match_type,info.outcome.winner,info.overs,info.teams,info.venue,match_id
0,"[{'1st innings': {'team': 'West Indies', 'deli...",Canberra,female,T20,Pakistan,20,"[Pakistan, West Indies]",Manuka Oval,1
1,"[{'1st innings': {'team': 'India', 'deliveries...",Bangalore,female,T20,India,20,"[India, Bangladesh]",M Chinnaswamy Stadium,2
2,"[{'1st innings': {'team': 'West Indies', 'deli...",Lauderhill,male,T20,India,20,"[India, West Indies]",Central Broward Regional Park Stadium Turf Ground,3
3,"[{'1st innings': {'team': 'West Indies', 'deli...",St Lucia,male,T20,Australia,20,"[West Indies, Australia]","Beausejour Stadium, Gros Islet",4
4,"[{'1st innings': {'team': 'India', 'deliveries...",Bangalore,male,T20,India,20,"[India, England]",M Chinnaswamy Stadium,5
...,...,...,...,...,...,...,...,...,...
1427,"[{'1st innings': {'team': 'Afghanistan', 'deli...",Chattogram,male,T20,Zimbabwe,20,"[Afghanistan, Zimbabwe]",Zahur Ahmed Chowdhury Stadium,1428
1428,"[{'1st innings': {'team': 'Namibia', 'deliveri...",Arbroath,female,T20,Netherlands,20,"[Namibia, Netherlands]",Lochlands,1429
1429,"[{'1st innings': {'team': 'Canada', 'deliverie...",Hamilton,male,T20,Canada,20,"[Canada, United States of America]","White Hill Field, Sandys Parish",1430
1430,"[{'1st innings': {'team': 'Pakistan', 'deliver...",Auckland,male,T20,New Zealand,20,"[Pakistan, New Zealand]",Eden Park,1431


In [8]:
# Remove female matches

final_df['info.gender'].value_counts()

info.gender
male      966
female    466
Name: count, dtype: int64

In [9]:
final_df = final_df[final_df['info.gender'] == 'male']
final_df.drop(columns = ['info.gender'], inplace = True)
final_df

/var/folders/cc/87njkkv159l3hskj13tfcpxr0000gn/T/ipykernel_6195/481311220.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.drop(columns = ['info.gender'], inplace = True)


,innings,info.city,info.match_type,info.outcome.winner,info.overs,info.teams,info.venue,match_id
2,"[{'1st innings': {'team': 'West Indies', 'deli...",Lauderhill,T20,India,20,"[India, West Indies]",Central Broward Regional Park Stadium Turf Ground,3
3,"[{'1st innings': {'team': 'West Indies', 'deli...",St Lucia,T20,Australia,20,"[West Indies, Australia]","Beausejour Stadium, Gros Islet",4
4,"[{'1st innings': {'team': 'India', 'deliveries...",Bangalore,T20,India,20,"[India, England]",M Chinnaswamy Stadium,5
5,"[{'1st innings': {'team': 'England', 'deliveri...",Nottingham,T20,South Africa,20,"[England, South Africa]",Trent Bridge,6
6,"[{'1st innings': {'team': 'South Africa', 'del...",Cape Town,T20,England,20,"[South Africa, England]",Newlands,7
...,...,...,...,...,...,...,...,...
1423,"[{'1st innings': {'team': 'Hong Kong', 'delive...",Abu Dhabi,T20,Oman,20,"[Hong Kong, Oman]",Sheikh Zayed Stadium,1424
1424,"[{'1st innings': {'team': 'Maldives', 'deliver...",Kuala Lumpur,T20,Thailand,20,"[Maldives, Thailand]",Kinrara Academy Oval,1425
1427,"[{'1st innings': {'team': 'Afghanistan', 'deli...",Chattogram,T20,Zimbabwe,20,"[Afghanistan, Zimbabwe]",Zahur Ahmed Chowdhury Stadium,1428
1429,"[{'1st innings': {'team': 'Canada', 'deliverie...",Hamilton,T20,Canada,20,"[Canada, United States of America]","White Hill Field, Sandys Parish",1430


In [10]:
final_df

,innings,info.city,info.match_type,info.outcome.winner,info.overs,info.teams,info.venue,match_id
2,"[{'1st innings': {'team': 'West Indies', 'deli...",Lauderhill,T20,India,20,"[India, West Indies]",Central Broward Regional Park Stadium Turf Ground,3
3,"[{'1st innings': {'team': 'West Indies', 'deli...",St Lucia,T20,Australia,20,"[West Indies, Australia]","Beausejour Stadium, Gros Islet",4
4,"[{'1st innings': {'team': 'India', 'deliveries...",Bangalore,T20,India,20,"[India, England]",M Chinnaswamy Stadium,5
5,"[{'1st innings': {'team': 'England', 'deliveri...",Nottingham,T20,South Africa,20,"[England, South Africa]",Trent Bridge,6
6,"[{'1st innings': {'team': 'South Africa', 'del...",Cape Town,T20,England,20,"[South Africa, England]",Newlands,7
...,...,...,...,...,...,...,...,...
1423,"[{'1st innings': {'team': 'Hong Kong', 'delive...",Abu Dhabi,T20,Oman,20,"[Hong Kong, Oman]",Sheikh Zayed Stadium,1424
1424,"[{'1st innings': {'team': 'Maldives', 'deliver...",Kuala Lumpur,T20,Thailand,20,"[Maldives, Thailand]",Kinrara Academy Oval,1425
1427,"[{'1st innings': {'team': 'Afghanistan', 'deli...",Chattogram,T20,Zimbabwe,20,"[Afghanistan, Zimbabwe]",Zahur Ahmed Chowdhury Stadium,1428
1429,"[{'1st innings': {'team': 'Canada', 'deliverie...",Hamilton,T20,Canada,20,"[Canada, United States of America]","White Hill Field, Sandys Parish",1430


In [11]:
final_df['info.match_type'].value_counts()

info.match_type
T20    966
Name: count, dtype: int64

In [12]:
final_df['info.overs'].value_counts()

info.overs
20    963
50      3
Name: count, dtype: int64

In [13]:
final_df = final_df[final_df['info.overs'] == 20]
final_df.drop(columns= ['info.overs', 'info.match_type'], inplace= True)

/var/folders/cc/87njkkv159l3hskj13tfcpxr0000gn/T/ipykernel_6195/2117545534.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.drop(columns= ['info.overs', 'info.match_type'], inplace= True)


In [14]:
final_df

,innings,info.city,info.outcome.winner,info.teams,info.venue,match_id
2,"[{'1st innings': {'team': 'West Indies', 'deli...",Lauderhill,India,"[India, West Indies]",Central Broward Regional Park Stadium Turf Ground,3
3,"[{'1st innings': {'team': 'West Indies', 'deli...",St Lucia,Australia,"[West Indies, Australia]","Beausejour Stadium, Gros Islet",4
4,"[{'1st innings': {'team': 'India', 'deliveries...",Bangalore,India,"[India, England]",M Chinnaswamy Stadium,5
5,"[{'1st innings': {'team': 'England', 'deliveri...",Nottingham,South Africa,"[England, South Africa]",Trent Bridge,6
6,"[{'1st innings': {'team': 'South Africa', 'del...",Cape Town,England,"[South Africa, England]",Newlands,7
...,...,...,...,...,...,...
1423,"[{'1st innings': {'team': 'Hong Kong', 'delive...",Abu Dhabi,Oman,"[Hong Kong, Oman]",Sheikh Zayed Stadium,1424
1424,"[{'1st innings': {'team': 'Maldives', 'deliver...",Kuala Lumpur,Thailand,"[Maldives, Thailand]",Kinrara Academy Oval,1425
1427,"[{'1st innings': {'team': 'Afghanistan', 'deli...",Chattogram,Zimbabwe,"[Afghanistan, Zimbabwe]",Zahur Ahmed Chowdhury Stadium,1428
1429,"[{'1st innings': {'team': 'Canada', 'deliverie...",Hamilton,Canada,"[Canada, United States of America]","White Hill Field, Sandys Parish",1430


In [15]:
pickle.dump(final_df, open('dataset_level1.pkl', 'wb'))

In [16]:
matches = pickle.load(open('dataset_level1.pkl', 'rb'))
matches

,innings,info.city,info.outcome.winner,info.teams,info.venue,match_id
2,"[{'1st innings': {'team': 'West Indies', 'deli...",Lauderhill,India,"[India, West Indies]",Central Broward Regional Park Stadium Turf Ground,3
3,"[{'1st innings': {'team': 'West Indies', 'deli...",St Lucia,Australia,"[West Indies, Australia]","Beausejour Stadium, Gros Islet",4
4,"[{'1st innings': {'team': 'India', 'deliveries...",Bangalore,India,"[India, England]",M Chinnaswamy Stadium,5
5,"[{'1st innings': {'team': 'England', 'deliveri...",Nottingham,South Africa,"[England, South Africa]",Trent Bridge,6
6,"[{'1st innings': {'team': 'South Africa', 'del...",Cape Town,England,"[South Africa, England]",Newlands,7
...,...,...,...,...,...,...
1423,"[{'1st innings': {'team': 'Hong Kong', 'delive...",Abu Dhabi,Oman,"[Hong Kong, Oman]",Sheikh Zayed Stadium,1424
1424,"[{'1st innings': {'team': 'Maldives', 'deliver...",Kuala Lumpur,Thailand,"[Maldives, Thailand]",Kinrara Academy Oval,1425
1427,"[{'1st innings': {'team': 'Afghanistan', 'deli...",Chattogram,Zimbabwe,"[Afghanistan, Zimbabwe]",Zahur Ahmed Chowdhury Stadium,1428
1429,"[{'1st innings': {'team': 'Canada', 'deliverie...",Hamilton,Canada,"[Canada, United States of America]","White Hill Field, Sandys Parish",1430


In [17]:
len(matches.iloc[0]['innings'])

2

# Extract delivery by delivery data

In [18]:
matches.iloc[0]['innings'][0]['1st innings']['deliveries']

[{0.1: {'non_striker': 'E Lewis',
   'bowler': 'Washington Sundar',
   'runs': {'extras': 0, 'total': 0, 'batsman': 0},
   'batsman': 'JD Campbell'}},
 {0.2: {'non_striker': 'E Lewis',
   'bowler': 'Washington Sundar',
   'runs': {'extras': 0, 'total': 0, 'batsman': 0},
   'batsman': 'JD Campbell',
   'wicket': {'player_out': 'JD Campbell',
    'fielders': ['KH Pandya'],
    'kind': 'caught'}}},
 {0.3: {'non_striker': 'E Lewis',
   'bowler': 'Washington Sundar',
   'runs': {'extras': 0, 'total': 0, 'batsman': 0},
   'batsman': 'N Pooran'}},
 {0.4: {'non_striker': 'E Lewis',
   'bowler': 'Washington Sundar',
   'runs': {'extras': 0, 'total': 1, 'batsman': 1},
   'batsman': 'N Pooran'}},
 {0.5: {'non_striker': 'N Pooran',
   'bowler': 'Washington Sundar',
   'runs': {'extras': 0, 'total': 0, 'batsman': 0},
   'batsman': 'E Lewis'}},
 {0.6: {'non_striker': 'N Pooran',
   'bowler': 'Washington Sundar',
   'runs': {'extras': 5, 'total': 5, 'batsman': 0},
   'extras': {'wides': 5},
   'batsm

In [19]:
count = 1
delivery_df = pd.DataFrame()
for index, row in matches.iterrows():
    if count in [75,108,150,180,268,360,443,458,584,748,982,1052,1111,1226,1345]:
        count+=1
        continue
    count+=1
    ball_of_match = []
    batsman = []
    bowler = []
    runs = []
    player_of_dismissed = []
    teams = []
    batting_team = []
    match_id = []
    city = []
    venue = []
    for ball in row['innings'][0]['1st innings']['deliveries']:
        for key in ball.keys():
            match_id.append(count)
            batting_team.append(row['innings'][0]['1st innings']['team'])
            teams.append(row['info.teams'])
            ball_of_match.append(key)
            batsman.append(ball[key]['batsman'])
            bowler.append(ball[key]['bowler'])
            runs.append(ball[key]['runs']['total'])
            city.append(row['info.city'])
            venue.append(row['info.venue'])
            try:
                player_of_dismissed.append(ball[key]['wicket']['player_out'])
            except:
                player_of_dismissed.append('0')
    loop_df = pd.DataFrame({
            'match_id':match_id,
            'teams':teams,
            'batting_team':batting_team,
            'ball':ball_of_match,
            'batsman':batsman,
            'bowler':bowler,
            'runs':runs,
            'player_dismissed':player_of_dismissed,
            'city':city,
            'venue':venue
})
    # Use pd.concat to append the loop_df DataFrame to delivery_df
    delivery_df = pd.concat([delivery_df, loop_df], ignore_index=True)
    # Removed erroneous line: df = pd.concat([df, new_df])

In [20]:
## Features to be used
# batting_team, Bowling_team, current_score, wickets_left, crr, city,

In [21]:
def bowl(row):
    for team in row['teams']:
        if team != row['batting_team']:
            return team

# def bowl(row):
#     teams = row['teams']
#     if not teams:  # This checks for both None and empty lists
#         return None
#     for team in teams:
#         if team != row['batting_team']:
#             return team

In [22]:
delivery_df['bowling_team'] = delivery_df.apply(bowl, axis=1)


In [23]:
delivery_df.drop(columns=['teams'], inplace=True)

In [24]:
delivery_df

,match_id,batting_team,ball,batsman,bowler,runs,player_dismissed,city,venue,bowling_team
0,2,West Indies,0.1,JD Campbell,Washington Sundar,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,India
1,2,West Indies,0.2,JD Campbell,Washington Sundar,0,JD Campbell,Lauderhill,Central Broward Regional Park Stadium Turf Ground,India
2,2,West Indies,0.3,N Pooran,Washington Sundar,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,India
3,2,West Indies,0.4,N Pooran,Washington Sundar,1,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,India
4,2,West Indies,0.5,E Lewis,Washington Sundar,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,India
...,...,...,...,...,...,...,...,...,...,...
115309,964,Pakistan,19.2,Shaheen Shah Afridi,BM Tickner,1,0,Auckland,Eden Park,New Zealand
115310,964,Pakistan,19.3,Faheem Ashraf,BM Tickner,0,Faheem Ashraf,Auckland,Eden Park,New Zealand
115311,964,Pakistan,19.4,Shaheen Shah Afridi,BM Tickner,2,0,Auckland,Eden Park,New Zealand
115312,964,Pakistan,19.5,Shaheen Shah Afridi,BM Tickner,1,0,Auckland,Eden Park,New Zealand


In [25]:
teams = [
    'Australia',
    'India',
    'Bangladesh',
    'New Zealand',
    'South Africa',
    'England',
    'West Indies',
    'Afghanistan',
    'Pakistan',
    'Sri Lanka'
]

In [26]:
delivery_df

,match_id,batting_team,ball,batsman,bowler,runs,player_dismissed,city,venue,bowling_team
0,2,West Indies,0.1,JD Campbell,Washington Sundar,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,India
1,2,West Indies,0.2,JD Campbell,Washington Sundar,0,JD Campbell,Lauderhill,Central Broward Regional Park Stadium Turf Ground,India
2,2,West Indies,0.3,N Pooran,Washington Sundar,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,India
3,2,West Indies,0.4,N Pooran,Washington Sundar,1,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,India
4,2,West Indies,0.5,E Lewis,Washington Sundar,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,India
...,...,...,...,...,...,...,...,...,...,...
115309,964,Pakistan,19.2,Shaheen Shah Afridi,BM Tickner,1,0,Auckland,Eden Park,New Zealand
115310,964,Pakistan,19.3,Faheem Ashraf,BM Tickner,0,Faheem Ashraf,Auckland,Eden Park,New Zealand
115311,964,Pakistan,19.4,Shaheen Shah Afridi,BM Tickner,2,0,Auckland,Eden Park,New Zealand
115312,964,Pakistan,19.5,Shaheen Shah Afridi,BM Tickner,1,0,Auckland,Eden Park,New Zealand


In [27]:
delivery_df = delivery_df[delivery_df['batting_team'].isin(teams)]
delivery_df = delivery_df[delivery_df['bowling_team'].isin(teams)]

In [28]:
delivery_df

,match_id,batting_team,ball,batsman,bowler,runs,player_dismissed,city,venue,bowling_team
0,2,West Indies,0.1,JD Campbell,Washington Sundar,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,India
1,2,West Indies,0.2,JD Campbell,Washington Sundar,0,JD Campbell,Lauderhill,Central Broward Regional Park Stadium Turf Ground,India
2,2,West Indies,0.3,N Pooran,Washington Sundar,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,India
3,2,West Indies,0.4,N Pooran,Washington Sundar,1,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,India
4,2,West Indies,0.5,E Lewis,Washington Sundar,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,India
...,...,...,...,...,...,...,...,...,...,...
115309,964,Pakistan,19.2,Shaheen Shah Afridi,BM Tickner,1,0,Auckland,Eden Park,New Zealand
115310,964,Pakistan,19.3,Faheem Ashraf,BM Tickner,0,Faheem Ashraf,Auckland,Eden Park,New Zealand
115311,964,Pakistan,19.4,Shaheen Shah Afridi,BM Tickner,2,0,Auckland,Eden Park,New Zealand
115312,964,Pakistan,19.5,Shaheen Shah Afridi,BM Tickner,1,0,Auckland,Eden Park,New Zealand


In [29]:
output = delivery_df[['match_id', 'batting_team', 'bowling_team', 'ball', 'runs', 'player_dismissed', 'city', 'venue']]

In [30]:
output

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,West Indies,India,0.1,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground
1,2,West Indies,India,0.2,0,JD Campbell,Lauderhill,Central Broward Regional Park Stadium Turf Ground
2,2,West Indies,India,0.3,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground
3,2,West Indies,India,0.4,1,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground
4,2,West Indies,India,0.5,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground
...,...,...,...,...,...,...,...,...
115309,964,Pakistan,New Zealand,19.2,1,0,Auckland,Eden Park
115310,964,Pakistan,New Zealand,19.3,0,Faheem Ashraf,Auckland,Eden Park
115311,964,Pakistan,New Zealand,19.4,2,0,Auckland,Eden Park
115312,964,Pakistan,New Zealand,19.5,1,0,Auckland,Eden Park


In [31]:
pickle.dump(output, open('dataset_level2.pkl', 'wb'))